##Imports

In [1]:
import numpy as np
import pandas as pd

##Loading Data

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/sajit9285/DNA-Classification-Project/master/promoters.data")

In [3]:
data.columns = ["class", "id", "sequence"]

In [4]:
data.head()

,class,id,sequence
0,+,AMPC,\t\ttgctatcctgacagttgtcacgctgattggtgtcgttacaat...
1,+,AROH,\t\tgtactagagaactagtgcattagcttatttttttgttatcat...
2,+,DEOP2,\taattgtgatgtgtatcgaagtgtgttgcggagtagatgttagaa...
3,+,LEU1_TRNA,\ttcgataattaactattgacgaaaagctgaaaaccactagaatgc...
4,+,MALEFG,\taggggcaaggaggatggaaagaggttgccgtataaagaaactag...


##Preprocessing and label encoding

In [5]:
classes = data["class"].tolist()

In [6]:
sequences = data["sequence"].tolist()

In [7]:
encoded_classes = []
for each in classes:
    if each == "+":
        encoded_classes.append(1)
    else:
        encoded_classes.append(0)


In [8]:
filtered_sequences = []
for each_sequence in sequences:
    filtered_sequences.append([each_char for each_char in each_sequence if each_char != "\t"])

In [9]:
index = [i for i in range(len(filtered_sequences[0]))]

In [10]:
dataframe = pd.DataFrame(filtered_sequences)

In [11]:
dataframe.columns =  index

In [12]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,t,g,c,t,a,t,c,c,t,g,a,c,a,g,t,t,g,t,c,a,c,g,c,t,g,a,t,t,g,g,t,g,t,c,g,t,t,a,c,a,a,t,c,t,a,a,c,g,c,a,t,c,g,c,c,a,a
1,g,t,a,c,t,a,g,a,g,a,a,c,t,a,g,t,g,c,a,t,t,a,g,c,t,t,a,t,t,t,t,t,t,t,g,t,t,a,t,c,a,t,g,c,t,a,a,c,c,a,c,c,c,g,g,c,g
2,a,a,t,t,g,t,g,a,t,g,t,g,t,a,t,c,g,a,a,g,t,g,t,g,t,t,g,c,g,g,a,g,t,a,g,a,t,g,t,t,a,g,a,a,t,a,c,t,a,a,c,a,a,a,c,t,c
3,t,c,g,a,t,a,a,t,t,a,a,c,t,a,t,t,g,a,c,g,a,a,a,a,g,c,t,g,a,a,a,a,c,c,a,c,t,a,g,a,a,t,g,c,g,c,c,t,c,c,g,t,g,g,t,a,g
4,a,g,g,g,g,c,a,a,g,g,a,g,g,a,t,g,g,a,a,a,g,a,g,g,t,t,g,c,c,g,t,a,t,a,a,a,g,a,a,a,c,t,a,g,a,g,t,c,c,g,t,t,t,a,g,g,t


##One hot encoding

In [13]:
encoded_df = pd.get_dummies(dataframe)


In [14]:
encoded_df.head()

,0_a,0_c,0_g,0_t,1_a,1_c,1_g,1_t,2_a,2_c,2_g,2_t,3_a,3_c,3_g,3_t,4_a,4_c,4_g,4_t,5_a,5_c,5_g,5_t,6_a,6_c,6_g,6_t,7_a,7_c,7_g,7_t,8_a,8_c,8_g,8_t,9_a,9_c,9_g,9_t,...,47_a,47_c,47_g,47_t,48_a,48_c,48_g,48_t,49_a,49_c,49_g,49_t,50_a,50_c,50_g,50_t,51_a,51_c,51_g,51_t,52_a,52_c,52_g,52_t,53_a,53_c,53_g,53_t,54_a,54_c,54_g,54_t,55_a,55_c,55_g,55_t,56_a,56_c,56_g,56_t
0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
3,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


##Machine Learning Classification Techniques

I have used three machine learning techniques to train the model 
1. K Nearest Neighbor classifier
2. Linear SVM Classifier
3. Kernal SVM

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

In [16]:
data = encoded_df.values

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
svm_linear = svm.SVC(kernel = 'linear')
svm_kernel = svm.SVC(kernel = 'rbf')

In [19]:
knn = KNeighborsClassifier(n_neighbors = 4)
from sklearn.model_selection import KFold, cross_val_score

def cross_validation_training(model, x, y):
    folds = KFold(n_splits = 10, shuffle= True)
    cross_validation = cross_val_score(model, x, y, cv = folds, scoring = 'accuracy')
    return cross_validation.mean()

Accuracy of three model 

In [20]:
svm_linear_cv_score = cross_validation_training(svm_linear, data , encoded_classes)


In [21]:
svm_kernel_cv_score = cross_validation_training(svm_kernel, data , encoded_classes)

In [22]:
knn_cv_score = cross_validation_training(knn, data , encoded_classes)

In [23]:
results = [svm_linear_cv_score, svm_kernel_cv_score, knn_cv_score]
results

[0.9318181818181819, 0.9154545454545454, 0.8372727272727273]

In [24]:
x_train, x_test, y_train, y_test = train_test_split(data, encoded_classes, test_size = 0.20, shuffle = True)

In [25]:
def model_evaluation(model, train_features, train_labels, test_features, test_labels):
    model.fit(train_features, train_labels)
    pred_test_labels = model.predict(test_features)
    print(accuracy_score(pred_test_labels, test_labels))
    print(classification_report(pred_test_labels, test_labels))
    

Priniting the recision, recall, f1-score, support of all the models

In [26]:
print("linear_svm_report")
model_evaluation(svm_linear, x_train, y_train, x_test, y_test)

linear_svm_report
0.8571428571428571
              precision    recall  f1-score   support

           0       0.90      0.82      0.86        11
           1       0.82      0.90      0.86        10

    accuracy                           0.86        21
   macro avg       0.86      0.86      0.86        21
weighted avg       0.86      0.86      0.86        21



In [27]:
print("kernel_svm_report")
model_evaluation(svm_kernel, x_train, y_train, x_test, y_test)

kernel_svm_report
0.8571428571428571
              precision    recall  f1-score   support

           0       1.00      0.77      0.87        13
           1       0.73      1.00      0.84         8

    accuracy                           0.86        21
   macro avg       0.86      0.88      0.86        21
weighted avg       0.90      0.86      0.86        21



In [28]:
print("KNN_report")
model_evaluation(knn, x_train, y_train, x_test, y_test)

KNN_report
0.8095238095238095
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        10
           1       0.82      0.82      0.82        11

    accuracy                           0.81        21
   macro avg       0.81      0.81      0.81        21
weighted avg       0.81      0.81      0.81        21



##Deep Learning Models
 I have used two Deep Learning models to train
 1. Simple Sequential Network
 2. Convolutional Neural Netwrok
 

In [29]:
import numpy
from keras import Input
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.losses import BinaryCrossentropy


In [30]:
x_train, x_test, y_train, y_test = train_test_split(data, encoded_classes, test_size = 0.20, shuffle = True)

In [31]:
model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [32]:
loss = BinaryCrossentropy()
model.compile(optimizer='adam', loss = loss, metrics=['accuracy'])

In [33]:
model.fit(x_train, np.array(y_train), epochs = 10, batch_size= 8)

Epoch 1/10
11/11 [==============================] - 0s 3ms/step - loss: 0.6990 - accuracy: 0.5000
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 0.5789 - accuracy: 0.9286
Epoch 3/10
11/11 [==============================] - 0s 2ms/step - loss: 0.2199 - accuracy: 0.9524
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 0.0334 - accuracy: 0.9881
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: 0.0274 - accuracy: 0.9762
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: 0.0171 - accuracy: 1.0000
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 0.0103 - accuracy: 0.9881
Epoch 8/10
11/11 [==============================] - 0s 2ms/step - loss: 3.2323e-04 - accuracy: 1.0000
Epoch 9/10
11/11 [==============================] - 0s 2ms/step - loss: 4.3601e-04 - accuracy: 1.0000
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: 3.2872e-04 - accuracy: 1.0000


Accuracy of simple sequential network

In [34]:
score = model.evaluate(x_test, np.array(y_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.6839235424995422 / Test accuracy: 0.8095238208770752


In [35]:
predicted_labels = model.predict(x_test)
test_labels = []
for each in predicted_labels:
  if each > 0.5:
    test_labels.append(1)
  else:
    test_labels.append(0)

print(classification_report(test_labels, np.array(y_test)))

              precision    recall  f1-score   support

           0       0.70      0.88      0.78         8
           1       0.91      0.77      0.83        13

    accuracy                           0.81        21
   macro avg       0.80      0.82      0.81        21
weighted avg       0.83      0.81      0.81        21



In [36]:
x_train.shape

(84, 228)

In [37]:
x_test.shape

(21, 228)

In [38]:
x_train = x_train.reshape(84,19,12,1)
x_test = x_test.reshape(21,19,12,1)

Convolutional Neural Network

In [39]:
from keras import datasets, layers, models
from keras.layers import LeakyReLU

In [40]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 2), activation='relu', input_shape=(19,12,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(100,activation=LeakyReLU(alpha=0.1)))
model.add(layers.Dense(100, activation=LeakyReLU(alpha=0.1)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1,activation="sigmoid"))

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 11, 32)        224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                81984     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               6500      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

In [42]:
loss = BinaryCrossentropy()
model.compile(optimizer='adam', loss = loss, metrics=['accuracy'])

In [43]:
model.fit(x_train, np.array(y_train), epochs = 20, batch_size= 8)

Epoch 1/20
11/11 [==============================] - 0s 5ms/step - loss: 0.6945 - accuracy: 0.5000
Epoch 2/20
11/11 [==============================] - 0s 3ms/step - loss: 0.6402 - accuracy: 0.7262
Epoch 3/20
11/11 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.7143
Epoch 4/20
11/11 [==============================] - 0s 3ms/step - loss: 0.3207 - accuracy: 0.9286
Epoch 5/20
11/11 [==============================] - 0s 3ms/step - loss: 0.1993 - accuracy: 0.9405
Epoch 6/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0702 - accuracy: 0.9881
Epoch 7/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0398 - accuracy: 1.0000
Epoch 8/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 9/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 10/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 11/20
11/11 [

Accuracy of Convolutional Neural Network

In [44]:

score = model.evaluate(x_test, np.array(y_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.28310495615005493 / Test accuracy: 0.8571428656578064


In [45]:
predicted_labels = model.predict(x_test)
test_labels = []
for each in predicted_labels:
  if each > 0.5:
    test_labels.append(1)
  else:
    test_labels.append(0)

print(classification_report(test_labels, np.array(y_test)))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87        13
           1       0.73      1.00      0.84         8

    accuracy                           0.86        21
   macro avg       0.86      0.88      0.86        21
weighted avg       0.90      0.86      0.86        21

